# Understanding geocoders

Geocoders are tools that can find spatial coordinates of addresses, business names, places of interest and so on. The output points can be visualized on a map, inserted as stops for a route, or loaded as input for spatial analysis. They also used to generate batch results for a set of addresses, as well as for reverse geocoding, i.e. determining the address at a particular x/y location.

The **`arcgis.geocoding`** module provides types and functions for geocoding, batch geocoding and reverse geocoding.

In this guide, we will observe
 - [The GIS's geocoders](#the-giss-geocoders)
 - [The default geocoder](#the-default-geocoder)
 - [Creating a geocoder using a geocoding service item](#creating-a-geocoder-using-a-geocoding-service-item)
 - [Creating a geocoder from a geocoding service](#creating-a-geocoder-from-a-geocoding-service)
 - [Using the default geocoder](#using-the-default-geocoder)
   - [Specifying a particular geocoder to be used](#specifying-a-particular-geocoder-to-be-used)
 - [Properties of a `geocoder`](#properties-of-a-geocoder)
   - [`addressField` property](#address-fields-property)
   - [`categories` property](#categories-property)
   - [`candidateFields` property](#candidate-fields-property)
   - [`spatialReference` property](#spatial-reference-property)
   - [`locator` properties](#locator-properties)

<a id="the-giss-geocoders"></a>
## The GIS's geocoders

A GIS includes one or more geocoders. The list of geocoders registered with the GIS can be queried using `get_geocoders()`. This method returns a list of `Geocoder` instances.

In the example below, there is one registered Geocoder with the GIS, that uses the [Esri World Geocoding Service](/features/geocoding/) for geocoding:

In [1]:
from arcgis.gis import GIS
from arcgis.geocoding import Geocoder, get_geocoders

gis = GIS("portal url", "username", "password")

get_geocoders(gis)

[<Geocoder url:"https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer">]

<a id="the-default-geocoder"></a>
## The default geocoder

The `arcgis.geocoding` modules provides functions for geocoding, reverse geocoding and batch geocoding. These functions use a Geocoder for their operation. All geocoding functions have an optional parameter for specifying the  geocoder to be used. 

The first available geocoder in the active GIS is used by default, unless specified.  Creating a new GIS object makes it’s first available geocoder as the default geocoder unless `set_active=False` is passed in the GIS constructor.

However, it is possible to use a different Geocoder by specifying it explicitly as a method parameter.

<a id="creating-a-geocoder-using-a-geocoding-service-item"></a>
## Creating a geocoder using a geocoding service item
Geocoding services can be published as items in the GIS. An instance of the geocoder can also be constructed by passing in a reference to these items from the GIS to the Geocoder's constructor:

In [2]:
from IPython.display import display

arcgis_online = GIS()
items = arcgis_online.content.search('Geocoder', 'geocoding service', max_items=3)
for item in items: 
    display(item)
    
# construct a geocoder using the first geocoding service item
worldgeocoder = Geocoder.fromitem(items[0])
worldgeocoder    

<Item title:"World Geocoding" type:Geocoding Service owner:esri>

<Item title:"Esri Nederland BAG Geocoder" type:Geocoding Service owner:Esri_NL_Content>

<Item title:"VGIN Composite Geocoder" type:Geocoding Service owner:VGIN>

<Geocoder url:"http://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer">

<a id="creating-a-geocoder-from-a-geocoding-service"></a>
## Creating a geocoder from a geocoding service

Geocoders may also be created using the constructor by passing in their location, such as a url to a Geocoding Service. If the geocoding service is a secure service, pass in the GIS to which it is federated with as the gis parameter:

In [3]:
geocoder_url = 'https://services.arcgisonline.nl/arcgis/rest/services/Geocoder_BAG_RD/GeocodeServer'
esrinl_geocoder = Geocoder(geocoder_url, gis)
esrinl_geocoder

<Geocoder url:"https://services.arcgisonline.nl/arcgis/rest/services/Geocoder_BAG_RD/GeocodeServer">

<a id="using-the-default-geocoder"></a>
## Using default geocoder

The example below shows calling the `geocode()` function to geocode an address. It does not pass in the geocoder to be used for geocoding, which makes the active GIS's first geocoder to be used by default.

In [4]:
from arcgis.geocoding import geocode

In [5]:
results = geocode('New York St, Redlands, CA')
# query the first matched result
results[0]['location']

{'x': -117.19568625471618, 'y': 34.054890875214824}

<a id="specifying-a-particular-geocoder-to-be-used"></a>
### Specifying the geocoder to be used

The example below shows calling the `geocode()` function to geocode an address. It specifies that the `esrinl_geocoder` created above should be used for geocoding by passing it in explicitly.

In [6]:
results = geocode('Raadhuisstraat 52, 1016 Amsterdam',  geocoder=esrinl_geocoder)
# query the first matched result
results[0]['location']

{'x': 120842.00295538208, 'y': 487472.9997233086, 'z': 0}

<hr>
<a id="properties-of-a-geocoder"></a>
## Properties of a `geocoder`

Geocoders have several properties and they can be accessed on `geocoder.properties` as shown in the intellisence below

![geocoder properties](http://esri.github.io/arcgis-python-api/notebooks/nbimages/guide_tools_geocoder_01.png)

The code snippet below lists the properties of the default geocoder:

In [7]:
geocoder = get_geocoders(gis)[0]
[prop_name for prop_name in geocoder.properties.keys()]

['currentVersion',
 'spatialReference',
 'serviceDescription',
 'capabilities',
 'locatorProperties',
 'addressFields',
 'categories',
 'candidateFields',
 'countries',
 'singleLineAddressField']

Some of the important properties are described in detail below:

<a id="address-fields-property"></a>
### `addressFields` property
The Geocoder's 'addressFields' property specifies the various address fields accepted by it when geocoding addresses.

For instance, the address fields accepted by this geocoder, and their length, are the following:

In [8]:
for addrfld in geocoder.properties.addressFields:
    print(addrfld['name'] + " (" + str(addrfld['length']) +" chars)")

Address (100 chars)
Neighborhood (50 chars)
City (50 chars)
Subregion (50 chars)
Region (50 chars)
Postal (20 chars)
PostalExt (20 chars)
CountryCode (100 chars)


#### Single Line Address Field
The geocoder may also support a single line address field. Single field input is easier because the address parsing is done for you; however, multifield input may provide faster responses and more precise results. The field name can be found using the code below:

In [9]:
geocoder.properties.singleLineAddressField['name']

'SingleLine'

When using single line input for the address, it is unnecessary (though supported) to create a dict with this key and the single line address as it's value. The address can be passed in directly as a text string.

One instance of when you might use a dict to include the SingleLine parameter is when it is combined with the countryCode parameter. The SingleLine parameter cannot be used with any of the other multifield parameters.

#### Localized input field names
Developers integrating the geocoder into their application may need to know the appropriate input field names to use for the language and country of their users. This information can be obtained using the 'localizedNames' parameter of the address field. More information on this as well as the 'recognizedNames' parameter is avilable in the <a href="/rest/geocode/api-reference/geocoding-localized-input-field-names.htm#ESRI_SECTION1_420C0A38D36546EDB1FC114048757B3E">Foreign language field names (World Geocoding Service)</a> documentation

For example, the code below lists the supported address fields and the corresponding input field names in Hindi:

In [10]:
for addrfld in geocoder.properties.addressFields:
    print(addrfld['name'], end='')
    print(": " + str(addrfld['localizedNames']['hi'] if 'hi' in addrfld['localizedNames'] else '-'))

Address: प्रमाचार
Neighborhood: इलाका
City: सिटी
Subregion: जनपद
Region: राष्ट्र
Postal: डाक - संबंधी
PostalExt: -
CountryCode: कंट्री कोड


<a id="categories-property"></a>
### `Categories` property
	
The 'categories' property can be used to limit result to one or more categories. For example, "Populated Place" or "Scandinavian Food". Only applies to the World Geocode Service. See <a href="/rest/geocode/api-reference/geocoding-category-filtering.htm#ESRI_SECTION1_502B3FE2028145D7B189C25B1A00E17B">Category filtering (World Geocoding Service)</a> for more information.

The following code lists the entire hierarchy of supported category values.

In [11]:
def list_categories(obj, depth = 0):       
    for category in obj['categories']:
        print('\t'*depth  + category['name'])
        if 'categories' in category:
            list_categories(category, depth + 1)
            
list_categories(geocoder.properties)

Address
	Point Address
	Building Name
	Street Address
	Intersection
	Street Name
	Distance Marker
Postal
	Primary Postal
	Postal Locality
	Postal Extension
Coordinate System
	LatLong
	MGRS
	XY
	YX
	USNG
Populated Place
	Neighborhood
	City
	Subregion
	Region
	Country
	Zone
POI
	Arts and Entertainment
		Amusement Park
		Aquarium
		Art Gallery
		Art Museum
		Billiards
		Bowling Alley
		Casino
		Cinema
		Historical Monument
		History Museum
		Indoor Sports
		Jazz Club
		Landmark
		Live Music
		Museum
		Other Arts and Entertainment
		Performing Arts
		Ruin
		Science Museum
		Tourist Attraction
		Wild Animal Park
		Zoo
	Education
		College
		Fine Arts School
		Other Education
		School
		Vocational School
	Food
		African Food
		American Food
		Argentinean Food
		Australian Food
		Austrian Food
		Bakery
		BBQ and Southern Food
		Belgian Food
		Bistro
		Brazilian Food
		Breakfast
		Brewpub
		British Isles Food
		Burgers
		Cajun and Creole Food
		Californian Food
		Caribbean Food
		Chicken Resta

<a id="candidate-fields-property"></a>
### `candidateFields` property
The CandidateFields property of the geocoder contains the fields that are returned for each candidate.

In [12]:
for addrfld in geocoder.properties.candidateFields:
    print(addrfld['name'])

Loc_name
Shape
Score
Match_addr
Addr_type
Type
PlaceName
Place_addr
Phone
URL
Rank
AddBldg
AddNum
AddNumFrom
AddNumTo
Side
StPreDir
StPreType
StName
StType
StDir
StAddr
Nbrhd
City
Subregion
Region
Postal
PostalExt
Country
LangCode
Distance
X
Y
DisplayX
DisplayY
Xmin
Xmax
Ymin
Ymax


<a id="spatial-reference-property"></a>
### `spatialReference` property
The default spatial reference returned by the geocoder. The code below is querying the well known id of the default spatial reference of the geocoder.

In [13]:
geocoder.properties.spatialReference['wkid']

4326

<a id="locator-properties"></a>
### Locator properties
The geocoder has several important properties that are specified in the locatorProperties. 

These include the maximum number of addresses that can be geocoded in a single batch geocoding method call. The MaxBatchSize property defines this limit. For instance, if MaxBatchSize=2000, and 3000 addresses are passed in as input to the batch_geocode() method, only the first 2000 will be geocoded. 

The SuggestedBatchSize property is also useful as it specifies the optimal number of addresses to include in a single batch request.

The code below lists these useful locator properties.

In [14]:
for key, value in geocoder.properties.locatorProperties.items():
    print(key + " : "+ str(value))

MaxBatchSize : 1000
WritePercentAlongField : FALSE
WriteReferenceIDField : FALSE
SuggestedBatchSize : 150
UICLSID : {3D486637-6BCF-4A0C-83DB-A02D437FB8FC}
WriteStandardizedAddressField : FALSE
IntersectionConnectors : &amp; @ | and
WriteXYCoordFields : TRUE
LoadBalancerTimeOut : 60
